##### The Directory Structire

```
dataset/
├── images/
│   ├── image1.jpg
│   ├── image2.jpg
│   └── ...
└── labels/
    ├── image1.txt
    ├── image2.txt
    └── ...

```

In [1]:
# This function takes 3 directory name and augment all the images in a directory
# Directory 1: input image folder (in_image_dir)
# Directory 2: input label folder (in_label_dir)
# Directory 3: output folder (out_dir)

def agment_images_in_dir(in_image_dir, in_label_dir, out_dir, extr_extension):

    # loop through each images and their corrosponding labels in specified dir
    for num, filename in enumerate(os.listdir(in_image_dir), start=1):

        # base name of a single image file without the extension 
        base_filename, image_extension = (os.path.splitext(filename)[0], os.path.splitext(filename)[1])
        
        # read the image file as numpy array
        image = cv2.imread(os.path.join(in_image_dir, filename))
        
        # filename of the corrosponding label file for the image above
        label_file = os.path.join(in_label_dir, base_filename + '.txt')

        # open the corrosponding label file
        with open(label_file, 'r') as f:
            label_data = f.readlines()

        # retrive the bounding boxes
        boxes = []
        for line in label_data:
            line = line.strip().split(' ')
            cls_id = int(line[0])
            x = float(line[1])
            y = float(line[2])
            w = float(line[3])
            h = float(line[4])
            boxes.append([x, y, w, h, cls_id])

        # Main augmentation happens here
        transformed = transform(image=image, bboxes=boxes)
        transformed_image = transformed['image']
        transformed_bboxes = transformed['bboxes']

        # Write augmented image to the disk
        cv2.imwrite(f'{os.path.join(out_dir, "images")}/{base_filename}_{extr_extension}_{num}{image_extension}', transformed_image)
        
        # Populate the name of the text file same as the image file
        output_label_file = os.path.join(out_dir, 'labels', f'{base_filename}_{extr_extension}_{num}' + '.txt')

        # Write yolo formatted text file to the disk
        with open(output_label_file, 'w') as f:
            for box in transformed_bboxes:
                f.write('%d %.6f %.6f %.6f %.6f\n' % (box[4], box[0], box[1], box[2], box[3]))

In [2]:
import albumentations as A
import cv2, os
import matplotlib.pyplot as plt

In [107]:
# Augmentation combination for stomata detection task

# Comb 1: ZoomBlur(max_factor=1.05, p=1)
    # transform = A.Compose([A.ZoomBlur(max_factor=1.05, p=1)], bbox_params=A.BboxParams(format='yolo'))

# Comb 2: HorizontalFlip(p=1) + Channel_shuffle
# transform = A.Compose([A.HorizontalFlip(p=1),A.ChannelShuffle(p=1)], bbox_params=A.BboxParams(format='yolo'))

# Comb 3: VerticalFlip(p=1) + ColorJitter(always_apply=True, p=1)
# transform = A.Compose([ A.VerticalFlip(p=1),A.ColorJitter(always_apply=True, p=1)], bbox_params=A.BboxParams(format='yolo'))

# Comb 4: Downscale
# transform = A.Compose([ A.Downscale(always_apply=True, p=1),], bbox_params=A.BboxParams(format='yolo'))

# Comb 5: RandomBrightnessContrast
# transform = A.Compose([A.RandomBrightnessContrast(always_apply=True, p=1),], bbox_params=A.BboxParams(format='yolo'))

# Comb 6: ISONoise(color_shift=(0.4, 0.6), p=1)
# transform = A.Compose([ A.ISONoise(color_shift=(0.4, 0.6), p=1),], bbox_params=A.BboxParams(format='yolo'))

# Comb 7: RandomRain(drop_length=10, blur_value=5, rain_type='drizzle', p=1)
# transform = A.Compose([A.RandomRain(drop_length=10, blur_value=5, rain_type='drizzle', p=1)], bbox_params=A.BboxParams(format='yolo'))

# Comb 8: A.RandomFog(fog_coef_lower=0.3, fog_coef_upper=0.5, always_apply=True, p=1)
# transform = A.Compose([A.RandomFog(fog_coef_lower=0.3, fog_coef_upper=0.5, always_apply=True, p=1)], bbox_params=A.BboxParams(format='yolo'))

# Comb 9: ToGray
# transform = A.Compose([A.ToGray(p=1)], bbox_params=A.BboxParams(format='yolo'))

# Comb 10: CLAHE(clip_limit=2.5, always_apply=True, p=1)
transform = A.Compose([A.CLAHE(clip_limit=2.5, always_apply=True, p=1)], bbox_params=A.BboxParams(format='yolo'))

# Comb 11: Resize (306, 306, interpolation=1, always_apply=False, p=1)
# transform = A.Compose([A.Resize (1224, 1224, interpolation=1, always_apply=True, p=1),], bbox_params=A.BboxParams(format='yolo'))

# Comb 12: LongestMaxSize + Transpose
    #   A.LongestMaxSize (max_size=2448, interpolation=1, always_apply=True, p=1), A.Transpose(p=1)


# transform = A.Compose([
#         A.Downscale(always_apply=True, p=1)
#     ], bbox_params=A.BboxParams(format='yolo'))

# transform = A.Compose([
#     A.Resize (306, 306, interpolation=1, always_apply=True, p=1),

# ], bbox_params=A.BboxParams(format='yolo'))

# Image augmentation pipeline 

# transform = A.Compose([
#     A.HorizontalFlip(p=1),
#     A.Blur(blur_limit=7, p=1),
#     A.ColorJitter(p=1),
#     A.Downscale(always_apply=True, p=1),
#     A.RandomBrightnessContrast(p=1),
# ], bbox_params=A.BboxParams(format='yolo'))



In [108]:
# Define the input and output directories
input_dir = 'dataset/Full_Image/val/images'
label_dir = 'dataset/Full_Image/val/labels'
output_dir = 'full_image_augmented_dataset/val'

# Create the output directories if the directory does not exists
os.makedirs(os.path.join(output_dir, 'images'), exist_ok=True)
os.makedirs(os.path.join(output_dir, 'labels'), exist_ok=True)

In [109]:
# Call the augmentation function for the actual augmentation to happen
# Function arguments: 
#       --> input_dir, 
#       --> label_dir, 
#       --> output_dir,
#       --> 4th arg: the extra extantion for filename. I.e., for the text "grey" will create the image and label name appended with 
#           _gray at the tailing part of the image and label. Ex: "amu_1_01_01_gray_199.txt"

agment_images_in_dir(input_dir, label_dir, output_dir, 'com_10')